In [72]:
import pandas as pd
import networkx as nx
import datetime
import matplotlib.pyplot as plt

In [73]:
df = pd.read_csv('fourCollections.csv', dtype={'collection_slug': 'str', 'asset_id': 'int', 'asset_name': 'str', 'owner_username': 'str', 'owner_address': 'str', 'event_type': 'str'})

In [74]:
# Clean Data
df.drop('Unnamed: 0.1', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('collection_slug', axis=1,inplace=True)
df.drop('asset_contract_date', axis=1, inplace=True)
df['event_timestamp'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%m/%d/%Y %H:%M')
df.drop_duplicates(keep='first')
df = df[df['event_type'] == 'successful']
df.head()

# Date Filter
df = df[(df['event_timestamp'] > '01/01/2022 1:00') & (df['event_timestamp'] < '01/07/2022 1:00')]

In [75]:
# Sorts addresses with more than 1 asset

az = 'Azuki'
ba = 'Bored Ape Yacht Club'
mf = 'mfers'
cc = 'Crypto Coven'

a = df[df['owner_address'].isin(df['owner_address'].value_counts()[df['owner_address'].value_counts() > 1].index)].owner_address.unique()

df_a = pd.DataFrame()
df_b = pd.DataFrame()

i = 0
for owner in a:
    temp = df[df['owner_address'] == a[i]]

    temp.reset_index(inplace=True)
    address = temp.owner_address[0]

    df_a = df_a.append(temp)

    mf_count = (temp[temp['collection_name'] == 'mfers'].count())
    m = mf_count['collection_name']

    cc_count = (temp[temp['collection_name'] == 'Crypto Coven'].count())
    c = cc_count['collection_name']

    if (m == 2 & c == 1):
        weight = '001'
    elif (c == 3):
        weight = '002'
    else:
        weight = '003'

    size = temp.shape[0] * 100
        

    f = {'address': address, 'weight': weight, 'size': size}
    df_b = df_b.append(f, ignore_index=True)
    
    i += 1

In [76]:
df_b

,address,weight,size
0,0x4ad5bf75e1ca307a4eb5cbb98f695c323e6362c4,003,200.0
1,0xb5bd745316567ba7784f16e5532fd57ac92772b7,003,200.0
2,0x0ed1e02164a2a9fad7a9f9b5b9e71694c3fad7f2,003,200.0
3,0xf142d7baff0986b50ae24e694419c65e7091f52c,003,200.0
4,0x6423c03e5f64d2537637f926907d05c0ae7714ec,003,200.0
...,...,...,...
158,0x2ecaf6b220f6b1a09a79397592fa569fda534637,003,600.0
159,0x6906041503264a221b8735186056ad89f7a4f04c,003,2300.0
160,0x600530afdccc8a765d3905c6d06cfd13504577ab,003,200.0
161,0x038a2c957a470373ea4ecf91493bcbf2b4a2ec93,003,200.0


In [77]:
'''
let's define nodes and links: each address is a node, the more total nfts owned from all collection the larger the node; 
if two address co-own any nfts from the same collection add a link with weight 1:
001 1 witch 2 mfers
002 3 witch
003 5 witch 1 mfers-
link1: 001-002, weight 1
link2: 001-003, weight 2
link3: 002-003, weight 1
'''

"\nlet's define nodes and links: each address is a node, the more total nfts owned from all collection the larger the node; \nif two address co-own any nfts from the same collection add a link with weight 1:\n001 1 witch 2 mfers\n002 3 witch\n003 5 witch 1 mfers-\nlink1: 001-002, weight 1\nlink2: 001-003, weight 2\nlink3: 002-003, weight 1\n"

In [78]:
G = nx.from_pandas_edgelist(df_b, 'address', 'weight')

In [82]:
edgelist = nx.to_edgelist(G)
edgelist

EdgeDataView([('0x4ad5bf75e1ca307a4eb5cbb98f695c323e6362c4', '003', {}), ('003', '0xb5bd745316567ba7784f16e5532fd57ac92772b7', {}), ('003', '0x0ed1e02164a2a9fad7a9f9b5b9e71694c3fad7f2', {}), ('003', '0xf142d7baff0986b50ae24e694419c65e7091f52c', {}), ('003', '0x6423c03e5f64d2537637f926907d05c0ae7714ec', {}), ('003', '0xeaddbae2651e21b4ae261ffbfcc81970854c4ef6', {}), ('003', '0xf896527c49b44aab3cf22ae356fa3af8e331f280', {}), ('003', '0xa54188a2e22766dbfc00a75a9f29b2b8fe93099e', {}), ('003', '0x82d30797cc1b191dcdebc6c2befe820ec8efe9cb', {}), ('003', '0xdbfd76af2157dc15ee4e57f3f942bb45ba84af24', {}), ('003', '0x42e7ca21c7212d3d1e8843671b5c64233e68168e', {}), ('003', '0x98f8f5ce4cd902f20ccadf3b6f12905fbe361391', {}), ('003', '0x4f3a120e72c76c22ae802d129f599bfdbc31cb81', {}), ('003', '0x93518c0a2ef7748e4de77ea7c98c0deab63dca95', {}), ('003', '0xf8c9cf133252a1a0b2c4b527381eaa94eaef62a6', {}), ('003', '0x51714e9e356045deaa392fcc502fa82bb45dd5c4', {}), ('003', '0xc56eb092b1ed018f093b72e29bb49c6

In [80]:
#plt.figure(figsize=(50,40))
#nx.draw(G, node_size='size')